In [323]:
import os
import json
import string
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
import pandas as pd
%matplotlib inline

In [324]:
def get_stream_details():
    print("Reading the stream details...")
    complete_stream_details_df = pd.read_csv("/Users/shriyashekhar/Documents/IWork/Data/tag_generation_content/card_module_details_content_extracted.csv", encoding="ISO-8859-1")
    if complete_stream_details_df is not None:
        complete_stream_details_dict = {}
        _stream_id_stream_title_dict = {}
        for _, row in complete_stream_details_df.iterrows():
            
            stream_id = str(row["DECKID"])
            stream_title = str(row["DECKNAME"])
            row_content = str(row["HTML_CONTENT"])

            # TODO: add the card title and the module name to the content on which the tags can be generated
            card_title =str(row["CARDTITLE"])
            module_name = str(row["MODULENAME"])
            
            if row_content and "nan" not in row_content:
                # if the stream ID already exists in the dictionary
                if complete_stream_details_dict.get(stream_id):
                    existing_content = complete_stream_details_dict[stream_id]
                    new_content = existing_content + "\n" + row_content.strip()
                    complete_stream_details_dict[stream_id] = new_content
                else:
                    complete_stream_details_dict[stream_id] = row_content.strip()
                    _stream_id_stream_title_dict[stream_id] = stream_title
        
        return complete_stream_details_dict

In [325]:
stream_details_dict = get_stream_details()

Reading the stream details...


In [326]:
df_ori = pd.DataFrame(list(stream_details_dict.items()), columns=["StreamID", "Content"])
df_ori.head()

,StreamID,Content
0,163,TXmAk2KZAy4\nNMeUjebo1Ac\nEEuTxFhp3go\nCastrol...
1,419,TXmAk2KZAy4\nNMeUjebo1Ac\nEEuTxFhp3go\nCastrol...
2,507,"wBYKUgUyGWc\nA team of world-class drivers, po..."
3,199,Castrol EDGE is Castrols flagship power bran...
4,201,"Charles Cheers Wakefield, Castrols founder,..."


# Pre-process the existing text data in the content

In [327]:
print(df_ori.shape)
df = df_ori.drop_duplicates(['Content'])
print(df.shape)
print("Removed {0} duplicates (based on Content)".format(df_ori.shape[0]-df.shape[0]))

(113, 2)
(97, 2)
Removed 16 duplicates (based on Content)


In [328]:
df["Content"].head()

0    TXmAk2KZAy4\nNMeUjebo1Ac\nEEuTxFhp3go\nCastrol...
1    TXmAk2KZAy4\nNMeUjebo1Ac\nEEuTxFhp3go\nCastrol...
2    wBYKUgUyGWc\nA team of world-class drivers, po...
3    Castrol EDGE  is Castrols flagship power bran...
4    Charles Cheers Wakefield, Castrols founder,...
Name: Content, dtype: object

In [329]:
df["Content"] = df["Content"].astype(str)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [330]:
def preprocess(tokens):
    tokens_nop = [t for t in tokens if t not in string.punctuation]
    tokens_nop = [t.lower() for t in tokens_nop]
    wnl = nltk.WordNetLemmatizer()
    stop = stopwords.words('english')
    tokens_nostop = [t for t in tokens_nop if t not in stop]
    tokens_lem = [wnl.lemmatize(t) for t in tokens_nostop]
    tokens_clean = [t for t in tokens_lem if len(t) >= 3]
    return tokens_clean


def plotWC(tokens):
    text_clean = " ".join(tokens)
    print(text_clean)
    wc = WordCloud(background_color="white").generate(text_clean)
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    plt.figure(figsize=(20, 9))
    fd = nltk.FreqDist(tokens)  # case sensitive!
    fd.plot(50)

## Shriya the below line has the method that you need to populate and use it in the chunk following

In [331]:
from nltk import ngrams
import nltk
nltk.download('wordnet')
def add_unique_bigrams(df):
    # step 1: remove the words constituting the bi grams from x and then
    # step 2: Add the BiGrams added at the end - below is just an example!!!
    all_streams_cleaned_tokens = []
    stream_ids = list(df["StreamID"])
#     print(df_ori["StreamID"])
    i=0
    combined = []
    for _, row in df.iterrows():
        stream_content = row["Content"]
        stream_content_tokens = nltk.word_tokenize(stream_content)
        stream_cleaned_tokens = preprocess(stream_content_tokens)
        all_streams_cleaned_tokens.append(stream_cleaned_tokens)
#         print(all_streams_cleaned_tokens)
        n_grams = ngrams(stream_cleaned_tokens, 3)

        for n_gram in n_grams:
            combined_val = "_".join(n_gram)
            combined.append(combined_val)
#     print(combined)
#     print("Original number of 3 tuples: " + str(len(combined)))
    combined = set(combined)
#     print("After duplicate removal: " + str(len(combined)))
#     df.extend(combined)
    return combined
    
    #     x.extend(["alpha_beta", "theta_zeta"])
    #     return x

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shriyashekhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [332]:
# x = ["a", "b","c"]

# y = add_unique_bigrams(x)
# x= add_unique_bigrams()
# print(x)
stream_1 = add_unique_bigrams(df)

print(stream_1)
stream_1 = set(stream_1)
print("After duplicate removal: " + str(len(stream_1)))


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: generator 'ngrams' raised StopIteration


{'status_navigate_user', 'titanium_fst\x99_continuously', 'affect_engine\x92s_lifespan', 'castrol_created_edge', '..._g9kvrnlsohk_castrol\x92s', 'minimum_indust_...', 'oil_drain_back', '..._viscosity_oil', 'genuine_please_reac', 'keep_engine_well', 'much_type_oil', 'want_share_exciting', 'performing_car_need', '..._permission_required', 'street_view_see', 'almost_doubled_engine', 'code_pack_foil', 'gtx_provides_better', 'demo_journey_plan', 'sludge_get_way', 'check_oil_level', 'preventing_oil_film', 'see_location_profit', 'around_car_engine', 'provide_continuous_protection', 'dealer_castrol_edge\x92s', 'intelligent_molecule_don\x92t', 'name_video_sequence', 'must_work_harder', 'work_pressure_reach', 'wakefield_pioneered_sponsorship', 'countless_stop_along', 'protection_high_low', 'earned_far_number', 'life_protecting_sludge', 'build-up_caused_everyday', 'use_catalytic_convert', 'mark_click_receive', 'redemption_acceptance_state', 'zoom_next_step', 'dirt_particle_join', 'redemption_payo

In [302]:
df['Content_token'] = df['Content'].map(word_tokenize)
df['Content_processed'] = df.Content_token.apply(preprocess)

# add the bigrams
df['Content_processed'] = df['Content_processed'].map(add_unique_bigrams)
df.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


TypeError: list indices must be integers or slices, not str

# Get the feature matrix

In [240]:
from sklearn.feature_extraction.text import TfidfVectorizer
df['Content_processedtext'] = df.Content_processed.apply(lambda x: ' '.join(x))

vec_tfidf = TfidfVectorizer(max_df=0.7, min_df=3, use_idf=True)
content_tfidf = vec_tfidf.fit_transform(df['Content_processedtext'])
print(content_tfidf.shape)

ModuleNotFoundError: No module named 'scipy'

# Reduce the dimensions for easier computation

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

# Use SVD to reduce dimensions to 50% features - try values between 25-50 for 70 to 100%
svd = TruncatedSVD(30)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X_lsa = lsa.fit_transform(content_tfidf)

In [36]:
explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

Explained variance of the SVD step: 81%


# Perform K means clustering

In [45]:
from sklearn.cluster import KMeans
from sklearn import metrics
num_clusters = 7

for num in [num_clusters]:
    km3 = KMeans(n_clusters=num, init='k-means++', max_iter=1000, n_init=1, random_state=1)
    %time km3.fit(X_lsa)
    # The higher the better (-1 to 1)
    print("Clusters: {0}".format(num))
    print("Silhouette Coefficient for clusters: %0.3f"
          % metrics.silhouette_score(X_lsa, km3.labels_))

Wall time: 5.98 ms
Clusters: 7
Silhouette Coefficient for clusters: 0.313


In [46]:
def print_terms(cm, num):
    original_space_centroids = svd.inverse_transform(cm.cluster_centers_)
    order_centroids = original_space_centroids.argsort()[:, ::-1]
    terms = vec_tfidf.get_feature_names()
    for i in range(num):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

print_terms(km3, num_clusters)

Cluster 0: wakefield curiosity smooth oil lubricant gear shift nasa power charles
Cluster 1: sequence field1 name type text page1 primaryfield header title card_description
Cluster 2: engine start magnatec protection warm part intelligent critical molecule cling
Cluster 3: sludge gtx engine oilways action formula dirt double keep life
Cluster 4: oil engine edge titanium strength pressure fst technology performance high
Cluster 5: happy would new year one like video see app work
Cluster 6: point redemption zoom app incentive click request transaction reward redeem


# Evaluate the co-occurences of different terms in the content

In [43]:
sparse_matrix = {}

for content_tokens in df['Content_processed']:
    for word1 in content_tokens:
        
        if not sparse_matrix.get(word1):
            sparse_matrix[word1] = {}

        for word2 in content_tokens:
            if sparse_matrix[word1].get(word2):
                sparse_matrix[word1][word2] += 1
            else:
                sparse_matrix[word1][word2] = 1

In [44]:
from collections import Counter
counts = Counter()
co_occurance_threshold = 1000
for word1 in sparse_matrix:
    for word2, co_occurence_count in sparse_matrix[word1].items():
        current_key_1 = "{0},{1}".format(word1, word2)
        current_key_2 = "{0},{1}".format(word2, word1)
        
        if word1 != word2 and co_occurence_count <= co_occurance_threshold:
            if counts.get(current_key_1):
                counts[current_key_1] += co_occurence_count // 2
            elif counts.get(current_key_2):
                counts[current_key_2] += co_occurence_count // 2
            else:
                counts[current_key_1] = co_occurence_count // 2
        
counts.most_common(50)

[('castrol,gtx', 978),
 ('engine,critical', 968),
 ('engine,warm-up', 918),
 ('...,point', 880),
 ('castrol,sludge', 860),
 ('oil,edge', 834),
 ('point,transaction', 824),
 ('product,point', 806),
 ('castrol,edge', 798),
 ('point,code', 790),
 ('engine,molecule', 778),
 ('oil,titanium', 774),
 ('...,incentive', 774),
 ('product,incentive', 730),
 ('incentive,transaction', 730),
 ('redemption,transaction', 728),
 ('engine,time', 726),
 ('castrol,point', 722),
 ('incentive,code', 710),
 ('engine,intelligent', 704),
 ('...,redemption', 694),
 ('point,click', 686),
 ('oil,pressure', 672),
 ('redemption,click', 666),
 ('product,redemption', 662),
 ('oil,strength', 656),
 ('engine,car', 650),
 ('code,redemption', 646),
 ('castrol,titanium', 642),
 ('castrol,part', 624),
 ('redemption,request', 624),
 ('point,request', 608),
 ('castrol,incentive', 592),
 ('point,user', 586),
 ('incentive,click', 584),
 ('engine,high', 580),
 ('castrol,strength', 578),
 ('engine,cling', 578),
 ('point,sale', 5